# Objective
The objetive of this notebook is to create models to forecast our cement production data. We will test different forecasting models, each with several hyperparameter configurations, and store results in MLflow.

# Description of the data
The dataset is quaterly cement production data.

# Imports, configuration and constants

In [27]:
import matplotlib.pyplot as plt
import mlflow
import pandas as pd

from mlflow.models import infer_signature
from prophet import Prophet
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from statsmodels.tsa.exponential_smoothing.ets import ETSModel
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

import utils as ut

In [2]:
import importlib
importlib.reload(ut)

<module 'utils' from '/home/santiagopalmero/repos/fpp3package/python/mlflow/utils.py'>

In [3]:
plt.rc("figure", figsize=(16, 12))
plt.rc("font", size=13)

In [4]:
PLOT_TITLE = "AUS cement production"
PLOT_YLABEL = "Cement"
PLOT_XLABEL = "Quarter"

PLOT_KWARGS = {
    "title": PLOT_TITLE,
    "ylabel": PLOT_YLABEL,
    "xlabel": PLOT_XLABEL,
}

# Load data

In [5]:
ts_train = ut.read_csv_series("data/ts_train.csv")
ts_test = ut.read_csv_series("data/ts_test.csv")

In [6]:
ts_train.index = pd.to_datetime(ts_train.index)
ts_train = ts_train.asfreq('QS-OCT')

ts_test.index = pd.to_datetime(ts_test.index)
ts_test = ts_test.asfreq('QS-OCT')

# Set experiment

In [7]:
mlflow.set_experiment("Cement_Forecasting")

<Experiment: artifact_location='file:///home/santiagopalmero/repos/fpp3package/python/mlflow/mlruns/570366757150847423', creation_time=1701436591943, experiment_id='570366757150847423', last_update_time=1701436591943, lifecycle_stage='active', name='Cement_Forecasting', tags={'mlflow.note.content': 'Project about cement production forecasting. This '
                        'experiment contains several forecasting models.',
 'project_name': 'forecasting'}>

In [ ]:
# One option that we have is to use autolog. I have tested and I dont dont find
# it very useful for this use case. I prefer to manually structure and log
# my models and metrics

# mlflow.statsmodels.autolog(
#     log_models=True,
#     disable=False,
#     exclusive=False,
#     disable_for_unsupported_versions=False,
#     silent=False,
#     registered_model_name=None,
# )

# Forecasting

## ARIMA

From the https://otexts.com/fpp3/arima-ets.html#comparing-arima-and-ets-on-seasonal-data section we know that the model that was used is `ARIMA(1,0,1)(2,1,1)[4] w/ drift`.

The drift concept is better explained in the previous version of the book https://otexts.com/fpp2/arima-r.html. In this link we can see an explanation about the drift in statsmodels https://stackoverflow.com/questions/66651360/arima-forecast-gives-different-results-with-new-python-statsmodels.

In [24]:
order = (1,0,1)
seasonal_order = (2,1,1,4)

arima = ARIMA(
    endog=ts_train, 
    order=order, 
    seasonal_order=seasonal_order, 
    # trend="t",
)
res = arima.fit()

ts_arima_h = res.predict(start=ts_test.index[0], end=ts_test.index[-1])

rmse = mean_squared_error(ts_test, ts_arima_h, squared=False)
mae = mean_absolute_error(ts_test, ts_arima_h)
mape = mean_absolute_percentage_error(ts_test, ts_arima_h)

In [25]:
with mlflow.start_run(run_name="arima"):
    mlflow.set_tag(
        "basic", 
        "Testing model development basic MLflow features.",
    )

    mlflow.log_params(
        {
            "order": order,
            "seasonal_order": arima.seasonal_order,
            "trend": arima.trend,
        }
    )
    mlflow.log_params(res.params)
    mlflow.log_params({"summary": res.summary()})

    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("mae", mae)
    mlflow.log_metric("mape", mape)

    model_info = mlflow.statsmodels.log_model(
        res, 
        artifact_path="ARIMA_model",
    )

/home/santiagopalmero/miniconda3/envs/fpp3/lib/python3.10/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/home/santiagopalmero/miniconda3/envs/fpp3/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


There are [differences in implementation](https://www.statsmodels.org/dev/examples/notebooks/generated/statespace_sarimax_faq.html#Differences-between-trend-and-exog-in-SARIMAX) between the class `ARIMA` and `SARIMAX`. So, lets try also this second one as the first one is not allowing as to add the constant term (drift).

In [36]:
sarimax = SARIMAX(
    endog=ts_train, 
    order=order, 
    seasonal_order=seasonal_order, 
    trend="c",
)
res = sarimax.fit()

ts_sarimax_h = res.predict(start=ts_test.index[0], end=ts_test.index[-1])

rmse = mean_squared_error(ts_test, ts_sarimax_h, squared=False)
mae = mean_absolute_error(ts_test, ts_sarimax_h)
mape = mean_absolute_percentage_error(ts_test, ts_sarimax_h)

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            7     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  5.93161D+00    |proj g|=  2.14605D-01

At iterate    5    f=  5.85403D+00    |proj g|=  3.80578D-02

At iterate   10    f=  5.85147D+00    |proj g|=  1.37646D-02


 This problem is unconstrained.



At iterate   15    f=  5.85027D+00    |proj g|=  3.39077D-02

At iterate   20    f=  5.82684D+00    |proj g|=  1.33501D-01

At iterate   25    f=  5.80155D+00    |proj g|=  5.88010D-03

At iterate   30    f=  5.79975D+00    |proj g|=  6.18598D-03

At iterate   35    f=  5.79736D+00    |proj g|=  3.61733D-03

At iterate   40    f=  5.79388D+00    |proj g|=  1.10395D-02

At iterate   45    f=  5.78620D+00    |proj g|=  2.96676D-02

At iterate   50    f=  5.78534D+00    |proj g|=  4.97323D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    7     50     63      1     0     0   4.973D-04   5.785D+00
  F =   5.78534418063

/home/santiagopalmero/miniconda3/envs/fpp3/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [37]:
with mlflow.start_run(run_name="sarimax"):
    mlflow.set_tag(
        "basic", 
        "Testing model development basic MLflow features.",
    )

    mlflow.log_params(
        {
            "order": order,
            "seasonal_order": sarimax.seasonal_order,
            "trend": sarimax.trend,
        }
    )
    mlflow.log_params(res.params)
    mlflow.log_params({"summary": res.summary()})

    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("mae", mae)
    mlflow.log_metric("mape", mape)

    model_info = mlflow.statsmodels.log_model(
        res, 
        artifact_path="SARIMAX_model",
    )

/home/santiagopalmero/miniconda3/envs/fpp3/lib/python3.10/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/home/santiagopalmero/miniconda3/envs/fpp3/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


As we can see in the MLflow gui this implementation matches the one of the book.

## ETS
From the books example in https://otexts.com/fpp3/arima-ets.html we can see the type of ETS model finally selected is ETS(M,N,M). From the R documentation https://www.rdocumentation.org/packages/forecast/versions/8.21/topics/ets we know that:
- The first letter denotes the error type ("A", "M" or "Z");
- The second letter denotes the trend type ("N","A","M" or "Z")
- The third letter denotes the season type ("N","A","M" or "Z").
- In all cases, "N"=none, "A"=additive, "M"=multiplicative and "Z"=automatically selected

In [18]:
ets = ETSModel(ts_train, error="mul", trend=None, seasonal="mul")
res = ets.fit()

ts_ets_h = res.predict(start=ts_test.index[0], end=ts_test.index[-1])

rmse = mean_squared_error(ts_test, ts_ets_h, squared=False)
mae = mean_absolute_error(ts_test, ts_ets_h)
mape = mean_absolute_percentage_error(ts_test, ts_ets_h)

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            6     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.91324D+00    |proj g|=  2.15029D+00

At iterate    1    f=  6.84699D+00    |proj g|=  1.65803D+00

At iterate    2    f=  6.42640D+00    |proj g|=  4.69153D-01

At iterate    3    f=  6.34069D+00    |proj g|=  4.86073D-01

At iterate    4    f=  6.26445D+00    |proj g|=  4.72329D-01

At iterate    5    f=  6.22885D+00    |proj g|=  3.81426D-01

At iterate    6    f=  6.18920D+00    |proj g|=  2.63642D-01

At iterate    7    f=  6.16398D+00    |proj g|=  4.70483D-01

At iterate    8    f=  6.15694D+00    |proj g|=  2.18301D-01

At iterate    9    f=  6.15418D+00    |proj g|=  9.72367D-02

At iterate   10    f=  6.15325D+00    |proj g|=  6.66425D-02

At iterate   11    f=  6.15227D+00    |proj g|=  6.25598D-02

At iterate   12    f=  6.14968D+00    |proj g|=  9.14247D-02

At iterate   13    f=  6.1

In [19]:
with mlflow.start_run(run_name="ets"):
    mlflow.set_tag(
        "basic", 
        "Testing model development basic MLflow features.",
    )

    mlflow.log_params(
        {
            "error": "mul",
            "trend": None,
            "seasonal": "mul",
        }
    )
    mlflow.log_params({"summary": res.summary()})

    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("mae", mae)
    mlflow.log_metric("mape", mape)

    model_info = mlflow.statsmodels.log_model(
        res, 
        artifact_path="ETS_model",
    )

/home/santiagopalmero/miniconda3/envs/fpp3/lib/python3.10/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/home/santiagopalmero/miniconda3/envs/fpp3/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


## Prophet
Again, we copy the model from the book https://otexts.com/fpp3/prophet.html.

In [38]:
df_train = (
    pd.DataFrame({
        'ds': ts_train.index.tolist(),
        'y': ts_train
    })
    .reset_index(drop=True)
)

In [39]:
m = Prophet()
m.add_seasonality(name='q', period=4, fourier_order=2, mode='multiplicative')
m.fit(df_train)

ts_pro_h, _, _ = ut.prophet_predict_w_interval(m, ts_test)

rmse = mean_squared_error(ts_test, ts_pro_h, squared=False)
mae = mean_absolute_error(ts_test, ts_pro_h)
mape = mean_absolute_percentage_error(ts_test, ts_pro_h)

17:17:19 - cmdstanpy - INFO - Chain [1] start processing
17:17:19 - cmdstanpy - INFO - Chain [1] done processing


In [52]:
with mlflow.start_run(run_name="prophet"):
    mlflow.set_tag(
        "basic", 
        "Testing model development basic MLflow features.",
    )

    mlflow.log_params(
        {
            "seasonality_q": "q",
            "seasonality_period": 4,
            "seasonality_fourier_order": 2,
            "seasonality_model": "multiplicative",
        }
    )

    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("mae", mae)
    mlflow.log_metric("mape", mape)

    # Instead of using df_train as input for infer_signature, we need to use the
    # the future df. If we dont do this, when we load the model we will need to 
    # create a df with the "y" column and this does not make sense for predict.
    future_df = m.make_future_dataframe(
        len(ts_test), 
        freq="QS-OCT", 
        include_history=False,
    )
    signature = infer_signature(future_df)

    model_info = mlflow.prophet.log_model(
        m, 
        artifact_path="Prophet_model",
        signature=signature,
        # input_example=df_train.head(),
    )

/home/santiagopalmero/miniconda3/envs/fpp3/lib/python3.10/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/home/santiagopalmero/miniconda3/envs/fpp3/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
